In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.4.2


In [2]:
!python -V

Python 3.10.14


In [3]:
import pickle
import pandas as pd

In [24]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/Users/rahulkalluri/Projects/mlops-zoomcamp/venv/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/rahulkalluri/Projects/mlops-zoomcamp/venv/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [25]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [30]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-05.parquet')

In [31]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

# Q1. Notebook

What's the standard deviation of the predicted duration for this dataset?

In [15]:
y_pred.std()

6.247488852238703

# Q2. Preparing the output

What's the size of the output file?

In [15]:
len(y_pred)

3316216

In [18]:
df.shape

(3316216, 21)

In [11]:
year = 2023
month = 3
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [19]:
df2 = df.assign(Predictions= y_pred)


(3316216, 22)

In [22]:
df_result = df2[['ride_id', 'Predictions']]

In [23]:
df_result.to_parquet(
    '/Users/rahulkalluri/Projects/mlops-zoomcamp/04-deployment/df_result.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)

# Q3. Creating the scoring script

In [32]:
import numpy as np
np.average(y_pred)

14.242595513316317